# Project Code

### ProjectCSGirls 2022

In [2]:
##import all of the things needed for the project code

import ST7735
from PIL import Image, ImageDraw

from IPython.display import clear_output, display, update_display
import smbus

import time

from bme280 import BME280
bme280 = BME280()
import busio
import board
import adafruit_ccs811

from board import *
i2c = board.I2C() 

import time
from datetime import datetime

Google Sheets Initial Code - Kristen

In [ ]:
## import the things necessary for Google Sheets in particular
import gspread
from oauth2client.service_account import ServiceAccountCredentials

sheet_name = "ProjectCSGirls"   # Make sure this matches your Google sheet name

# use creds to create a client to interact with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('projectcsgirls-345320-ec70b92cf651.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

Google Sheets Initial Code - Katie

In [4]:
## import the things necessary for Google Sheets in particular
import gspread
from oauth2client.service_account import ServiceAccountCredentials

sheet_name = "ProjectCSGirls"   # Make sure this matches your Google sheet name

# use creds to create a client to interact with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

APIError: {'code': 403, 'message': 'Google Sheets API has not been used in project 470861842948 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/sheets.googleapis.com/overview?project=470861842948 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'status': 'PERMISSION_DENIED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Google developers console API activation', 'url': 'https://console.developers.google.com/apis/api/sheets.googleapis.com/overview?project=470861842948'}]}, {'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'SERVICE_DISABLED', 'domain': 'googleapis.com', 'metadata': {'consumer': 'projects/470861842948', 'service': 'sheets.googleapis.com'}}]}

Testing each measurement individually

In [11]:
##temperature

## is it working?
## how to print??

def readTemp():
    # By default the address of LM75A is set to 0x48
    address = 0x48

    # Read I2C data and calculate temperature
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [10]:
## humidity

h = bme280.get_humidity()
print (h)

23.50598525775839


In [6]:
## CO2 and TVOC

ccs = adafruit_ccs811.CCS811(i2c)

print("CO2: ", ccs.eco2, "TVOC:", ccs.tvoc)

CO2:  0 TVOC: 0
